# Connection

> Helps Manage Snowflake Connection

In [1]:
#| default_exp functions

In [6]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from __future__ import annotations
from customfunctions.common import print_hello

In [ ]:
#| export


def hello_function(name: str) -> str:
    return print_hello(name)

In [15]:
#| hide
import nbdev; nbdev.nbdev_export()